## Model Evaluation 

In [81]:
# import dependencies 
import math 

In [66]:
# Define input variables  
groups = [[1, 2, 3], [1], [0, 2]] 
votes = [5, 5, 5, 5]

In [67]:
def group_memberships(groups, votes):
    """
    Define group memberships for each participant.
    :param: groups (list of lists): a list denotes the group and contains its members. 
    :param: votes (list): number of participant's votes for a given project proposal.
    :returns (list of lists): retunrs a list of group membersips for each participant.  
    """
    # let each entry in votes define a unique identifyer for an agent 
    group_memberships = [[] for _ in range(len(votes))]

    # add group memberships of agent i as a list to each entry identifying an agent 
    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)
        
    return group_memberships

In [68]:
group_memberships = group_memberships(groups, votes)
print(group_memberships)

[[2], [0, 1], [0, 2], [0]]


In [69]:
def common_group(agent_i, agent_j):
    """
    Define an identifyer indicating whether two participants share any common group. 
    :param: agent_i denotes a participant not equal to a participant called agent_j. 
    :param: agent_j denotes a participant not equal to a participant called agent_i.
    :returns (bool): returns true if two participants share a common group and false otherwise.  
    """
    common_group = any(group in group_memberships[agent_j] for group in group_memberships[agent_i])

    return common_group

In [70]:
def K(agent_i, group):
    """
    Define the weighting function that attenuates the votes of agent i given different group memberships.  
    :param: agent_i denotes a participant not equal to a participant called agent_j.
    :param: (integer): group denotes the group number.
    :returns: attenuated number of votes for a given project. 
    """
    if agent_i in group or any(common_group(agent_i, j) for j in group):
        return math.sqrt(votes[agent_i])
    else:
        return votes[agent_i]


In [79]:
def connection_oriented_cluster_match(groups, votes, group_memberships):
    """
    This function calculates the connection-oriented cluster match of votes 
    for each agent and returns the attenuated sum of votes for a given project proposal.
    param: groups (list of lists): a list denotes the group and contains its members.
    param: votes (list): number of participant's votes for a given project proposal.
    param: group_membership: defines group memberships for each participant.
    """ 
    weighted_votes = 0

    # Calculate the first part of the weighted votes
    weighted_votes += sum(votes[agent_i] / len(group_memberships[agent_i]) for group in groups for agent_i in group)

    for group in groups:
        for other_group in groups:
            if group == other_group:
                continue

            interaction_term1 = math.sqrt(sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group))
            interaction_term2 = math.sqrt(sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group))

            weighted_votes += interaction_term1 * interaction_term2
    
    sqrt_weighted_votes = math.sqrt(weighted_votes)

    return sqrt_weighted_votes

In [80]:
result = connection_oriented_cluster_match(groups, votes, group_memberships)
print(result)

6.119546254329342
